# Eco-Switch AI — index.ipynb
This notebook cleans datasets, trains a simple fuel-price predictor, and outputs cleaned files and predictions. Run cells in order.

In [1]:

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import LinearRegression
import joblib
import warnings
warnings.filterwarnings('ignore')
print("Libraries imported")


Libraries imported


In [2]:

# Paths for the datasets (adjust names if your files are different)
base = Path(".")
paths = {
    "fuel_master": base / r"C:\Users\Asus\OneDrive\Documents\Important\1M1B Green Internship\Project\Master_Fuel_Prices_2015_2024.csv",
    "ev_specs": base / r"C:\Users\Asus\OneDrive\Documents\Important\1M1B Green Internship\Project\EV Specs.csv",
    "gef": base / r"C:\Users\Asus\OneDrive\Documents\Important\1M1B Green Internship\Project\Grid Emission Factors (GEF).csv",
    "car_mileage": base / r"C:\Users\Asus\OneDrive\Documents\Important\1M1B Green Internship\Project\Fuel Car Mileage.csv",
    "co2_file": base / r"C:\Users\Asus\OneDrive\Documents\Important\1M1B Green Internship\Project\CO₂ per litre (petrol & diesel).csv"
}
for k,v in paths.items():
    print(k, "->", v.exists())


fuel_master -> True
ev_specs -> True
gef -> True
car_mileage -> True
co2_file -> True


In [3]:

# Show columns for each file if present
for name, p in paths.items():
    if p.exists():
        try:
            df = pd.read_csv(p)
        except Exception as e:
            try:
                df = pd.read_excel(p)
            except Exception as e2:
                print("Could not read", p)
                continue
        print("\n----", name, "columns ----")
        print(list(df.columns)[:40])
    else:
        print(name, "missing:", p)



---- fuel_master columns ----
['Country', 'Fuel Type', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

---- ev_specs columns ----
['brand', 'model', 'top_speed_kmh', 'battery_capacity_kWh', 'battery_type', 'number_of_cells', 'torque_nm', 'efficiency_wh_per_km', 'range_km', 'acceleration_0_100_s', 'fast_charging_power_kw_dc', 'fast_charge_port', 'towing_capacity_kg', 'cargo_volume_l', 'seats', 'drivetrain', 'segment', 'length_mm', 'width_mm', 'height_mm', 'car_body_type', 'source_url']

---- gef columns ----
['Country', 'Grid Intensity (kg CO₂/kWh)', 'Currency Code', 'Currency Symbol']

---- car_mileage columns ----
['Dimensions.Height', 'Dimensions.Length', 'Dimensions.Width', 'Engine Information.Driveline', 'Engine Information.Engine Type', 'Engine Information.Hybrid', 'Engine Information.Number of Forward Gears', 'Engine Information.Transmission', 'Fuel Information.City mpg', 'Fuel Information.Fuel Type', 'Fuel Information.Highway mpg', 'Identificati

In [4]:

# CLEAN: Master fuel prices (wide -> long)
p = paths["fuel_master"]
if p.exists():
    df = pd.read_csv(p)
    # Standardize column names
    df.columns = [c.strip() for c in df.columns]
    # Melt years into rows
    id_vars = ["Country", "Fuel Type"] if "Fuel Type" in df.columns else ["Country"]
    year_cols = [c for c in df.columns if c.strip().isdigit()]
    if not year_cols:
        # fallback: any column that looks like 2015..2024
        year_cols = [c for c in df.columns if any(ch.isdigit() for ch in c)]
    long = df.melt(id_vars=id_vars, value_vars=year_cols, var_name="year", value_name="price")
    long = long.dropna(subset=["price"])
    long["year"] = long["year"].astype(str).str.extract(r'(\d{4})')[0].astype(int)
    # Separate petrol and diesel
    if "Fuel Type" in long.columns:
        petrol = long[long["Fuel Type"].str.contains("petrol", case=False, na=False)].rename(columns={"price":"petrol_price"})
        diesel = long[long["Fuel Type"].str.contains("diesel", case=False, na=False)].rename(columns={"price":"diesel_price"})
        merged = pd.merge(petrol[["Country","year","petrol_price"]], diesel[["Country","year","diesel_price"]], on=["Country","year"], how="outer")
    else:
        # If file already has separate columns for petrol/diesel, attempt to pivot
        pivot = df.copy()
        # If petrol and diesel are columns
        if "petrol" in (c.lower() for c in pivot.columns) and "diesel" in (c.lower() for c in pivot.columns):
            # try to normalize names
            cols = {c:c for c in pivot.columns}
            # rename common variants
            rename_map = {}
            for c in pivot.columns:
                lc = c.lower()
                if "petrol" in lc or "gasoline" in lc:
                    rename_map[c] = "petrol_price"
                if "diesel" in lc and "price" not in lc:
                    rename_map[c] = "diesel_price"
            pivot = pivot.rename(columns=rename_map)
            # melt if years present
            if any(str(y) in pivot.columns for y in range(2015,2025)):
                melt_id = [c for c in pivot.columns if not str(2015) <= c <= str(2024)]
                long = pivot.melt(id_vars=melt_id, var_name="year", value_name="price")
                long["year"] = long["year"].astype(str).str.extract(r'(\d{4})')[0].astype(int)
                merged = long.copy()
            else:
                merged = pivot.copy()
        else:
            merged = long.copy()

    merged = merged.rename(columns={"Country":"country"})
    merged = merged.sort_values(["country","year"]).reset_index(drop=True)
    merged.to_csv("clean_fuel_prices.csv", index=False)
    print("Saved clean_fuel_prices.csv rows:", len(merged))
else:
    print("Fuel master file missing")


Saved clean_fuel_prices.csv rows: 1407


In [5]:

# CLEAN: EV specs
p = paths["ev_specs"]
if p.exists():
    df = pd.read_csv(p)
    df.columns = [c.strip() for c in df.columns]
    # pick useful columns (robust selection)
    col_map = {}
    for c in df.columns:
        lc = c.lower()
        if "brand" in lc:
            col_map[c] = "brand"
        if "model" in lc and "model year" not in lc:
            col_map[c] = "model"
        if "battery" in lc and "kwh" in lc:
            col_map[c] = "battery_kwh"
        if "efficiency" in lc and ("wh" in lc or "wh_per" in lc):
            col_map[c] = "efficiency_wh_per_km"
        if "range" in lc and "km" in lc:
            col_map[c] = "range_km"
    ev = df.rename(columns=col_map)
    # compute kwh per 100 km
    if "efficiency_wh_per_km" in ev.columns:
        ev["kwh_per_100km"] = (ev["efficiency_wh_per_km"] / 1000) * 100
    elif "battery_kwh" in ev.columns and "range_km" in ev.columns:
        ev["kwh_per_100km"] = ev["battery_kwh"] / (ev["range_km"] / 100)  # approximate
    else:
        ev["kwh_per_100km"] = None

    keep = [c for c in ["brand","model","battery_kwh","kwh_per_100km","range_km"] if c in ev.columns]
    clean_ev = ev[keep].copy()
    clean_ev.to_csv("clean_ev_specs.csv", index=False)
    print("Saved clean_ev_specs.csv rows:", len(clean_ev))
else:
    print("EV specs file missing")


Saved clean_ev_specs.csv rows: 478


In [6]:

# CLEAN: GEF
p = paths["gef"]
if p.exists():
    df = pd.read_csv(p)
    df.columns = [c.strip() for c in df.columns]
    # find GEF column
    gef_col = None
    for c in df.columns:
        if "grid" in c.lower() and ("co2" in c.lower() or "intensity" in c.lower()):
            gef_col = c
            break
    if gef_col is None:
        # try common name
        possible = [c for c in df.columns if "kg" in c.lower() and "co2" in c.lower()]
        gef_col = possible[0] if possible else df.columns[1]
    clean_gef = df.rename(columns={gef_col: "gef", df.columns[0]:"country"})[[ "country", "gef" ]].copy()
    # numeric
    clean_gef["gef"] = pd.to_numeric(clean_gef["gef"], errors="coerce")
    clean_gef.to_csv("clean_gef.csv", index=False)
    print("Saved clean_gef.csv rows:", len(clean_gef))
else:
    print("GEF file missing")


Saved clean_gef.csv rows: 124


In [7]:

# CLEAN: Car mileage (convert mpg to km/l if needed)
p = paths["car_mileage"]
if p.exists():
    df = pd.read_csv(p)
    df.columns = [c.strip() for c in df.columns]
    # Try to find fuel type and mpg columns
    fuel_col = None
    mpg_col = None
    for c in df.columns:
        lc = c.lower()
        if "fuel type" in lc or "fuelinformation.fuel type" in lc or "fuel information.fuel type" in lc:
            fuel_col = c
        if "mpg" in lc and ("city" in lc or "fuel information.city mpg" in lc or "city mpg" in lc):
            mpg_col = c
    # fallback search
    if mpg_col is None:
        for c in df.columns:
            if "mpg" in c.lower():
                mpg_col = c
                break
    if fuel_col is None:
        for c in df.columns:
            if "fuel" in c.lower() and "type" in c.lower():
                fuel_col = c
                break
    # build clean table
    if mpg_col is not None:
        clean_m = df[[c for c in [fuel_col, mpg_col] if c in df.columns]].copy()
        clean_m.columns = ["fuel_type","mpg"]
        # convert mpg -> km/l
        clean_m["km_per_litre"] = pd.to_numeric(clean_m["mpg"], errors="coerce") * 0.425144
        clean_m = clean_m.dropna(subset=["km_per_litre"])
        clean_m.to_csv("clean_mileage.csv", index=False)
        print("Saved clean_mileage.csv rows:", len(clean_m))
    else:
        print("Could not find mpg column in car mileage file. Saving a minimal table if possible.")
        df.to_csv("clean_mileage_raw.csv", index=False)
else:
    print("Car mileage file missing")


Saved clean_mileage.csv rows: 5076


In [ ]:

# CO2 per litre (petrol/diesel) - prefer user file, else use defaults
p = paths["co2_file"]
if p.exists():
    try:
        df = pd.read_csv(p)
        print("CO2 file columns:", list(df.columns)[:20])
        # If file contains fuel_type and co2_per_litre use directly, else fallback
        cols = [c.lower() for c in df.columns]
        if "fuel_type" in cols or "fuel type" in cols or "fuel" in cols:
            # try to standardize
            if "co2_per_litre" in cols or "co2_per_litre" in df.columns:
                df = df[["fuel_type","co2_per_litre"]]
            else:
                # attempt to map
                print("Using default constants because file format is unexpected")
                raise Exception("unexpected")
        else:
            raise Exception("unexpected")
    except Exception as e:
        clean_co2 = pd.DataFrame({"fuel_type":["Petrol","Diesel"], "co2_per_litre":[2.31,2.68]})
        clean_co2.to_csv("clean_co2.csv", index=False)
        print("Saved default clean_co2.csv")
else:
    clean_co2 = pd.DataFrame({"fuel_type":["Petrol","Diesel"], "co2_per_litre":[2.31,2.68]})
    clean_co2.to_csv("clean_co2.csv", index=False)
    print("Saved default clean_co2.csv")


CO2 file columns: ['Make', 'Model', 'Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission', 'Fuel Type', 'Fuel Consumption City (L/100 km)', 'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)', 'Fuel Consumption Comb (mpg)', 'CO2 Emissions(g/km)']
Using default constants because file format is unexpected
Saved default clean_co2.csv


In [ ]:

# TRAIN: Simple Linear Regression per country for petrol_price (if petrol present)
import os
from sklearn.linear_model import LinearRegression
import json

if Path("clean_fuel_prices.csv").exists():
    df = pd.read_csv("clean_fuel_prices.csv")
    # Ensure columns
    df.columns = [c.strip() for c in df.columns]
    # We'll attempt model per country for petrol_price and diesel_price if available
    preds = []
    countries = df['country'].dropna().unique().tolist()
    for country in countries:
        sub = df[df['country']==country].copy()
        for fuel in ['petrol_price','diesel_price']:
            if fuel in sub.columns and sub[fuel].notna().sum() >= 5:
                tmp = sub[['year', fuel]].dropna()
                X = tmp[['year']].values
                y = tmp[fuel].values
                model = LinearRegression()
                model.fit(X, y)
                future_years = np.arange(2025, 2031).reshape(-1,1)
                ypred = model.predict(future_years)
                for yr, val in zip(range(2025,2031), ypred):
                    preds.append({"country":country, "fuel":fuel, "year":int(yr), "predicted_price":float(val)})
                # save model per country+fuel
                model_filename = f"model_{country.replace(' ','_')}_{fuel}.pkl"
                try:
                    joblib.dump(model, model_filename)
                except Exception as e:
                    pass
    pred_df = pd.DataFrame(preds)
    pred_df.to_csv("fuel_price_predictions_2025_2030.csv", index=False)
    with open("fuel_price_predictions_2025_2030.json","w") as f:
        json.dump(preds, f, indent=2)
    print("Saved predictions:", len(preds))
else:
    print("clean_fuel_prices.csv missing; cannot train model")


Saved predictions: 1308


In [ ]:

import os
print("Files created in /mnt/data:")
for fname in ["clean_fuel_prices.csv","clean_ev_specs.csv","clean_gef.csv","clean_mileage.csv","clean_co2.csv","fuel_price_predictions_2025_2030.csv","fuel_price_predictions_2025_2030.json","index.ipynb"]:
    print(fname, "->", Path(fname).exists())


Files created in /mnt/data:
clean_fuel_prices.csv -> True
clean_ev_specs.csv -> True
clean_gef.csv -> True
clean_mileage.csv -> True
clean_co2.csv -> True
fuel_price_predictions_2025_2030.csv -> True
fuel_price_predictions_2025_2030.json -> True
index.ipynb -> True
